In [1]:
import sys
import logging
# logging.getLogger().setLevel(logging.ERROR) # Be quiet!!
# logging.disable(sys.maxsize) # Be quiet!!

import copy
import torch
import torch.nn as nn
from torch.utils.data import *
from transformers import *
import inspect
sys.path.insert(0, "..")

from models import *
from logic import *
from my_datasets import *

from utils import *
import numpy as np

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
n, r = 8, 20
# ap, bp, tp, sp = 0.2, 0.2, 0.4, 0.1
ap, bp, tp, sp = 0.2, 0.1, 0.2, 0.2

nars = 3

train_len = 1000
test_len = 500
num_epochs = 20
test_is_train = False

In [3]:
qed_training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    num_train_epochs = num_epochs,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 64,
    logging_steps = 5,
    do_train = True,
)

succ_training_args = qed_training_args
ars_training_args = qed_training_args

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [4]:
qed_mytf = get_task_model(task_name="oneshot_qed", num_vars=n, model_name="mytf")
succ_mytf = get_task_model(task_name="predict_successor", num_vars=n, model_name="mytf")
ars_mytf = get_task_model(task_name="autoreg_fixed_steps", num_vars=n, num_steps=3, model_name="mytf")

In [5]:
qed_gpt2 = get_task_model(task_name="oneshot_qed", num_vars=n, model_name="gpt2")
succ_gpt2 = get_task_model(task_name="predict_successor", num_vars=n, model_name="gpt2")
ars_gpt2 = get_task_model(task_name="autoreg_fixed_steps", num_vars=n, num_steps=nars, model_name="gpt2")

In [6]:
qed_bert = get_task_model(task_name="oneshot_qed", num_vars=n, model_name="bert")
succ_bert = get_task_model(task_name="predict_successor", num_vars=n, model_name="bert")
ars_bert = get_task_model(task_name="autoreg_fixed_steps", num_vars=n, num_steps=nars, model_name="bert")

In [7]:
### Datasets
qed_train_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=train_len,seed=1234)
qed_test_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=test_len,seed=2345)
qed_train_dataset = OneShotQedEmbedsDataset(qed_train_dataset_config)
qed_test_dataset = OneShotQedEmbedsDataset(qed_test_dataset_config)

succ_train_dataset_config = OneStepStateDatasetConfig(r,n,ap,bp,tp,dataset_len=train_len,seed=1234)
succ_test_dataset_config = OneStepStateDatasetConfig(r,n,ap,bp,tp,dataset_len=test_len,seed=2345)
succ_train_dataset = OneStepStateEmbedsDataset(succ_train_dataset_config)
succ_test_dataset = OneStepStateEmbedsDataset(succ_test_dataset_config)

ars_train_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,nars,dataset_len=train_len,seed=1234)
ars_test_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,nars,dataset_len=test_len,seed=2345)
ars_train_dataset = AutoRegFixedStepsEmbedsDataset(ars_train_dataset_config)
ars_test_dataset = AutoRegFixedStepsEmbedsDataset(ars_test_dataset_config)

In [21]:
### SUCC MyTF
trainer_succ_mytf = Trainer(succ_mytf, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_train_dataset if test_is_train else succ_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_succ_mytf.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 42,814,472


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.683100,0.694508,0.552000,0.875000
2,0.691500,0.691668,0.529000,0.750000
3,0.692200,0.691662,0.529000,0.750000
4,0.690000,0.687506,0.553500,0.875000
5,0.687600,0.685741,0.576500,1.000000
6,0.685200,0.683962,0.576500,1.000000
7,0.690100,0.683073,0.576500,1.000000
8,0.685700,0.682922,0.576500,1.000000
9,0.684500,0.682750,0.576500,1.000000
10,0.684300,0.682612,0.576500,1.000000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.6834647856652737, metrics={'train_runtime': 19.1224, 'train_samples_per_second': 1045.894, 'train_steps_per_second': 16.734, 'total_flos': 0.0, 'train_loss': 0.6834647856652737, 'epoch': 20.0})

In [9]:
### SUCC GPT2
trainer_succ_gpt2 = Trainer(succ_gpt2, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_train_dataset if test_is_train else succ_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_succ_gpt2.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 125,248,512


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.667100,0.690450,0.556250,0.604750
2,0.652400,0.634135,0.585500,0.835500
3,0.587800,0.603900,0.625500,0.840000
4,0.502900,0.551027,0.701500,0.649500
5,0.354600,0.523338,0.741750,0.522250
6,0.272300,0.536816,0.780250,0.442750
7,0.237700,0.578583,0.774250,0.448250
8,0.201100,0.717782,0.759250,0.450250
9,0.176000,0.580735,0.770250,0.523250
10,0.122600,0.755811,0.761750,0.472750


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.22835787981748581, metrics={'train_runtime': 454.9164, 'train_samples_per_second': 43.964, 'train_steps_per_second': 0.703, 'total_flos': 0.0, 'train_loss': 0.22835787981748581, 'epoch': 20.0})

In [10]:
### SUCC Bert
trainer_succ_bert = Trainer(succ_bert, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_train_dataset if test_is_train else succ_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_succ_bert.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 110,290,952


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.684900,0.695523,0.552000,0.875000
2,0.694500,0.688526,0.553500,0.875000
3,0.687000,0.683313,0.576500,1.000000
4,0.680100,0.682192,0.576500,1.000000
5,0.670000,0.682152,0.576500,1.000000
6,0.659200,0.682986,0.557500,0.869500
7,0.614700,0.659346,0.607250,0.751750
8,0.508100,0.601572,0.667000,0.473000
9,0.451900,0.587752,0.676000,0.678000
10,0.352500,0.504080,0.734500,0.538500


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.4067021680995822, metrics={'train_runtime': 27.5699, 'train_samples_per_second': 725.428, 'train_steps_per_second': 11.607, 'total_flos': 0.0, 'train_loss': 0.4067021680995822, 'epoch': 20.0})

In [11]:
####################
### AR Steps

In [20]:
### AR Steps MyTF
trainer_ars_mytf = Trainer(ars_mytf, ars_training_args,
    train_dataset = ars_train_dataset,
    eval_dataset = ars_train_dataset if test_is_train else ars_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_ars_mytf.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 42,814,472


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.639600,0.645606,0.614000,0.875000
2,0.657100,0.642743,0.660500,1.000000
3,0.647300,0.642112,0.660500,1.000000
4,0.645800,0.641187,0.660500,1.000000
5,0.641400,0.641380,0.660500,1.000000
6,0.641800,0.641278,0.660500,1.000000
7,0.654700,0.641324,0.660500,1.000000
8,0.646700,0.641314,0.660500,1.000000
9,0.645200,0.641319,0.660500,1.000000
10,0.641200,0.641324,0.660500,1.000000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.6409725308418274, metrics={'train_runtime': 42.0449, 'train_samples_per_second': 475.682, 'train_steps_per_second': 7.611, 'total_flos': 0.0, 'train_loss': 0.6409725308418274, 'epoch': 20.0})

In [19]:
### AR Steps GPT2
trainer_ars_gpt2 = Trainer(ars_gpt2, ars_training_args,
    train_dataset = ars_train_dataset,
    eval_dataset = ars_train_dataset if test_is_train else ars_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_ars_gpt2.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 125,248,512


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.630200,0.632283,0.602417,0.635083
2,0.644300,0.626365,0.641417,0.714917
3,0.622200,0.615171,0.654917,0.677083
4,0.606200,0.604872,0.657583,0.587083
5,0.574900,0.599474,0.694833,0.719167
6,0.557700,0.576041,0.731750,0.670750
7,0.547400,0.577125,0.740000,0.709167
8,0.530200,0.553980,0.772167,0.619167
9,0.517600,0.555047,0.740667,0.517667
10,0.502600,0.549649,0.752000,0.523000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.5259912237524986, metrics={'train_runtime': 1362.425, 'train_samples_per_second': 14.68, 'train_steps_per_second': 0.235, 'total_flos': 0.0, 'train_loss': 0.5259912237524986, 'epoch': 20.0})

In [14]:
### AR Steps Bert
trainer_ars_bert = Trainer(ars_bert, ars_training_args,
    train_dataset = ars_train_dataset,
    eval_dataset = ars_train_dataset if test_is_train else ars_test_dataset,
    compute_metrics = succ_compute_metrics)
trainer_ars_bert.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 110,290,952


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.637900,0.643999,0.571583,0.750583
2,0.655900,0.641426,0.660500,1.000000
3,0.646100,0.640557,0.660500,1.000000
4,0.635400,0.637052,0.668500,0.965167
5,0.628000,0.629701,0.613917,0.672250
6,0.625900,0.628242,0.677000,0.900000
7,0.629500,0.615823,0.650750,0.673750
8,0.604300,0.606695,0.696167,0.782000
9,0.594400,0.595508,0.708833,0.721667
10,0.568700,0.575134,0.736333,0.614500


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.5727014072239399, metrics={'train_runtime': 69.3641, 'train_samples_per_second': 288.333, 'train_steps_per_second': 4.613, 'total_flos': 0.0, 'train_loss': 0.5727014072239399, 'epoch': 20.0})

In [15]:
####################
### QED

In [16]:
### QED MyTf
trainer_qed_mytf = Trainer(qed_mytf, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_train_dataset if test_is_train else qed_test_dataset,
    compute_metrics = qed_compute_metrics)
trainer_qed_mytf.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 42,811,394


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.712600,0.692761,0.530000,1.000000
2,0.686700,0.697782,0.530000,1.000000
3,0.695300,0.692597,0.530000,1.000000
4,0.707200,0.691484,0.530000,1.000000
5,0.691700,0.709520,0.530000,1.000000
6,0.694300,0.694137,0.530000,1.000000
7,0.685700,0.692728,0.530000,1.000000
8,0.691000,0.695937,0.530000,1.000000
9,0.694400,0.706705,0.530000,1.000000
10,0.688900,0.705754,0.530000,1.000000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.6986616715788841, metrics={'train_runtime': 31.0719, 'train_samples_per_second': 643.669, 'train_steps_per_second': 10.299, 'total_flos': 0.0, 'train_loss': 0.6986616715788841, 'epoch': 20.0})

In [17]:
### QED GPT2
trainer_qed_gpt2 = Trainer(qed_gpt2, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_train_dataset if test_is_train else qed_test_dataset,
    compute_metrics = qed_compute_metrics)
trainer_qed_gpt2.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 125,243,904


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.807200,0.710974,0.530000,1.000000
2,0.667900,0.648395,0.600000,0.782000
3,0.584100,0.608090,0.682000,0.456000
4,0.495300,0.708883,0.674000,0.344000
5,0.453700,0.833598,0.598000,0.832000
6,0.556300,1.559920,0.556000,0.958000
7,0.185700,1.141354,0.678000,0.632000
8,0.072800,2.434420,0.646000,0.732000
9,0.127300,2.172845,0.700000,0.510000
10,0.011900,2.406397,0.676000,0.614000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.25327228133430424, metrics={'train_runtime': 466.7478, 'train_samples_per_second': 42.85, 'train_steps_per_second': 0.686, 'total_flos': 0.0, 'train_loss': 0.25327228133430424, 'epoch': 20.0})

In [18]:
### QED Bert
trainer_qed_bert = Trainer(qed_bert, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_train_dataset if test_is_train else qed_test_dataset,
    compute_metrics = qed_compute_metrics)
trainer_qed_bert.train()

***** Running training *****
  Num examples = 1,000
  Num Epochs = 20
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 110,286,338


Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.727900,0.722326,0.470000,0.000000
2,0.685800,0.706336,0.530000,1.000000
3,0.668700,0.702796,0.530000,1.000000
4,0.653900,0.697784,0.530000,1.000000
5,0.603300,0.766296,0.470000,0.000000
6,0.549400,0.697353,0.470000,0.000000
7,0.514400,1.265155,0.470000,0.000000
8,0.378000,1.259487,0.530000,1.000000
9,0.211800,1.959402,0.470000,0.000000
10,0.293900,1.287681,0.530000,1.000000


***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evaluation *****
  Num examples = 500
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=320, training_loss=0.27700417206797284, metrics={'train_runtime': 41.065, 'train_samples_per_second': 487.033, 'train_steps_per_second': 7.793, 'total_flos': 0.0, 'train_loss': 0.27700417206797284, 'epoch': 20.0})